In [3]:
import mlflow
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [2]:
mlflow.set_tracking_uri('http://3.7.252.23:5000')
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-mlflow-campusx/728060098183646235', creation_time=1731836197994, experiment_id='728060098183646235', last_update_time=1731836197994, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [5]:
df = pd.read_csv('dataset.csv').dropna()
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()

[I 2024-11-17 18:05:48,972] A new study created in memory with name: no-name-2a97c84c-0479-4877-9837-e6def9ba1f4e
[I 2024-11-17 18:05:49,025] Trial 0 finished with value: 0.66856901289368 and parameters: {'alpha': 0.0005347933807218382}. Best is trial 0 with value: 0.66856901289368.
[I 2024-11-17 18:05:49,056] Trial 1 finished with value: 0.66856901289368 and parameters: {'alpha': 0.00018793488703977028}. Best is trial 0 with value: 0.66856901289368.
[I 2024-11-17 18:05:49,081] Trial 2 finished with value: 0.6682519551891778 and parameters: {'alpha': 0.03173421582808639}. Best is trial 0 with value: 0.66856901289368.
[I 2024-11-17 18:05:49,110] Trial 3 finished with value: 0.6683576410906785 and parameters: {'alpha': 0.005056945898012899}. Best is trial 0 with value: 0.66856901289368.
[I 2024-11-17 18:05:49,146] Trial 4 finished with value: 0.66856901289368 and parameters: {'alpha': 0.0026864764628291744}. Best is trial 0 with value: 0.66856901289368.
[I 2024-11-17 18:05:49,187] Trial 